In [ ]:
import os

In [ ]:
os.chdir("drive/My Drive/Thesis")

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
from gensim.models import FastText

In [ ]:
pre_model = FastText.load_fasttext_format('w2v/wiki.simple')

In [ ]:
np.shape(pre_model['cat'])

NameError: ignored

In [ ]:
if 'In' in pre_model:
  print('Hello')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
import json
import string

In [ ]:
raw_text = []
raw_label = []
with open('training.json') as f:
  file = json.load(f)
  f.close()
for x in file:
  raw_text.append(x['text'])
  raw_label.append(x['label'])

In [ ]:
import tensorflow as tf

In [ ]:
sequence_length = 55 # tune it to max 300

def wv_input(text_list, label_list):
  input_id_list = []
  attention_mask_list = []
  tag_list = []
  for sentence, stripe in zip(text_list, label_list):
    temp_token = []
    temp_tag = []
    temp_attention = []
    temp_type = []
    for word, tag in zip(sentence.split(), stripe):
      if tag == 2:
        tag = 0
      if word in string.punctuation:
        continue
     # print('here')
     # print(word)
      if word in pre_model:
        new_token = [pre_model[word]]
        new_label = [tag] 
        new_attention = [1]
      else:
        new_token = [np.array([-1.0 for i in range(300)])]
        new_label = [tag]
        new_attention = [1]
      
      temp_token.extend(new_token)
      temp_tag.extend(new_label)
      temp_attention.extend(new_attention)
   ## print(np.shape(temp_token))
   # print(temp_token)


    if len(temp_token) < sequence_length:
      extend_list = [[-1.00 for i in range(300)]] * (sequence_length - len(temp_token))
      extend_tag = [2] * (sequence_length - len(temp_token))
      temp_attention = temp_attention + [0] * (sequence_length - len(temp_token))
      temp_token = temp_token + extend_list
      temp_tag = temp_tag + extend_tag

    elif len(temp_token) > sequence_length:
      temp_token = temp_token[:sequence_length ]
      temp_attention = temp_attention[:sequence_length]
      temp_tag = temp_tag[:sequence_length]

    input_id_list.append(temp_token)
    attention_mask_list.append(temp_attention)
    tag_list.append(temp_tag)

  
  
  return np.array(input_id_list), np.array(attention_mask_list), np.array(tag_list)

In [ ]:
import numpy as np

In [ ]:
input_data, mask_data, output_data  = wv_input(raw_text, raw_label)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
np.shape(input_data), np.shape(output_data)

((580, 100, 300), (580, 100))

In [ ]:
train_data_x = np.array(input_data[:464])
valid_data_x = np.array(input_data[464:])                   

In [ ]:
train_data_y = np.array(output_data[:464]).reshape(464, -1, 1)
valid_data_y = np.array(output_data[464:]).reshape(116, -1, 1)

In [ ]:
train_mask = np.array(mask_data[:464])
valid_mask = np.array(mask_data[464:])

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
best = 0
num_label = 3
def see(ty, tp):
  #model.load('')
  ty = ty.reshape(-1)
  tp = tp.reshape(-1, num_label)
  #print(tp.shape, ty.shape)
  tp = [np.argmax(x) for x in tp]
  dix = np.where(ty == (num_label - 1))[0]
  tp = np.delete(tp, dix)
  ty = np.delete(ty, dix)
  #valid_p = [1 if x > 0.5 else 0 for x in model.predict(valid_x)]
  #valid_p = model.predict(valid_x)
 # print(np.shape(tp), ty.shape)
  print('\nClassification Report \n\n')
  
  f = classification_report(ty, tp)
  print(f)
 # print('\nPrint Confusion Matrix \n\n')
  print(confusion_matrix(ty, tp))
  return f1_score(ty, tp, average=None)[1]

In [ ]:
class mcb(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, log=None):
   # global train_x, train_y, valid_x, valid_y
    print('here')
    _y = self.model.predict(train_data_x)
    see(train_data_y, _y)
    _y = self.model.predict(valid_data_x)
    see(valid_data_y, _y)

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.LSTM(units=512, return_sequences=True),
                             tf.keras.layers.LSTM(units=128, return_sequences=True),
                             tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(64, activation='relu')),
                             tf.keras.layers.Dense(1024, activation='relu'),
                             tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(train_data_x, train_data_y, epochs=3,
          validation_data=(valid_data_x, valid_data_y),
          sample_weight=train_mask, callbacks=[mcb()],
          batch_size=8)

Epoch 1/3
58/58 [==============================] - 37s 568ms/step - loss: 0.3806 - accuracy: 0.5320 - val_loss: 4.3631 - val_accuracy: 0.5792
here
(25520, 3) (25520,)
(25520,) (25520,)

Classification Report 


              precision    recall  f1-score   support

           0       0.57      1.00      0.72     14432
           1       0.93      0.01      0.02      2831
           2       0.00      0.00      0.00      8257

    accuracy                           0.57     25520
   macro avg       0.50      0.34      0.25     25520
weighted avg       0.42      0.57      0.41     25520


Print Confusion Matrix 


[[14430     2     0]
 [ 2806    25     0]
 [ 8257     0     0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(6380, 3) (6380,)
(6380,) (6380,)

Classification Report 


              precision    recall  f1-score   support

           0       0.58      1.00      0.73      3694
           1       0.67      0.00      0.00       833
           2       0.00      0.00      0.00      1853

    accuracy                           0.58      6380
   macro avg       0.42      0.33      0.25      6380
weighted avg       0.42      0.58      0.43      6380


Print Confusion Matrix 


[[3693    1    0]
 [ 831    2    0]
 [1853    0    0]]
Epoch 2/3
58/58 [==============================] - 31s 537ms/step - loss: 0.2457 - accuracy: 0.5860 - val_loss: 3.0334 - val_accuracy: 0.6005
here
(25520, 3) (25520,)
(25520,) (25520,)

Classification Report 


              precision    recall  f1-score   support

           0       0.58      1.00      0.73     14432
           1       0.88      0.18      0.30      2831
           2       0.00      0.00      0.00      8257

    accuracy                           0.58     